In [ ]:
!nvidia-smi

In [ ]:
#hugginface loggin:

from huggingface_hub import login

login(token="your huggingface login token")


In [ ]:
from huggingface_hub import whoami

# Check login
print(whoami())

In [ ]:
# Code template and prompt elements

prompt_prefix = """ 
### SYSTEM
Your task is to generate Unity C# code for a sword based on the user input. Follow the provided examples. Return only the new sword item code as describe in the examples.
###END

### USER INPUT:
"""

Sword_Item_Parameters = """
###Sword Parameters
The sword item has the following paramters which can be used: 
// Basic stats
    public float damage // the basic damage of the sword item
    public float swingSpeed // how fast the sword is 
    
    // Damage over time properties
    public bool hasDamageOverTime // if the sword has a damage over time effect like poison
    public float dotDamage // the damge per second of the effect
    public float dotDuration // how long the effect lasts
    
    // Elemental bonus damage properties
    public bool hasElementalDamage // a flat bonus damage value
    public float fireBonusDamage // bonus damage from fire
    public float iceBonusDamage // bonus damage from ice
    public float lightningBonusDamage // bonus damage from lighting or electricity 
    
    // Critical hit properties
    [Range(0f, 1f)]
    public float critChance // the chance to critical extra damage
    public float critMultiplier // critical hits multiple base damage 
    
    // Swing Arc or Range properties
    // swingArc can be used for visual/animation purposes, while colliderSize adjusts the area of effect.
    public float swingArc  // e.g., degrees of the swing arc
    public Vector2 colliderSize //adjusting collider size which should reflect the size of the sword

### END
"""


Sword_Code_Template = """
###SWORD BASE CLASS 
The base class that governs the use of the sword item is described below, ensure that the generated item block thus not results in errors when interacting with this code.


using UnityEngine;

public class SwordAttack : MonoBehaviour
{
    public Collider2D swordCollider;
    public SwordData swordData; // Reference to the data-driven properties for this sword
    
    // Store the original offset of the sword
    Vector2 rightAttackOffset;

    private void Start()
    {
        rightAttackOffset = transform.localPosition;
    }

    public void AttackRight()
    {
        if (swordData == null)
        {
            Debug.LogWarning("SwordData not assigned!");
            return;
        }
        
        // Optionally adjust the collider size if using a BoxCollider2D
        if (swordCollider is BoxCollider2D box)
        {
            box.size = swordData.colliderSize;
        }

        swordCollider.enabled = true;
        transform.localPosition = rightAttackOffset;
        SwingAnimation(swordData.swingSpeed, swordData.swingArc);
    }

    public void AttackLeft()
    {
        if (swordData == null)
        {
            Debug.LogWarning("SwordData not assigned!");
            return;
        }

        if (swordCollider is BoxCollider2D box)
        {
            box.size = swordData.colliderSize;
        }
        
        swordCollider.enabled = true;
        transform.localPosition = new Vector3(-rightAttackOffset.x, rightAttackOffset.y);
        SwingAnimation(swordData.swingSpeed, swordData.swingArc);
    }

    public void StopAttack()
    {
        swordCollider.enabled = false;
    }

    private void SwingAnimation(float speed, float arc)
    {
        // Example: You can trigger an animation using speed and arc
        Debug.Log("Swinging sword at speed: " + speed + " with arc: " + arc);
    }

    private void OnTriggerEnter2D(Collider2D other)
    {
        if (other.CompareTag("Enemy") && swordData != null)
        {
            Enemy enemy = other.GetComponent<Enemy>();
            if (enemy != null)
            {
                // Calculate base damage
                float totalDamage = swordData.damage;

                // Apply elemental bonus damage (summed up for this example)
                totalDamage += swordData.fireBonusDamage;
                totalDamage += swordData.iceBonusDamage;
                totalDamage += swordData.lightningBonusDamage;

                // Determine if a critical hit occurs
                if (Random.value <= swordData.critChance)
                {
                    totalDamage *= swordData.critMultiplier;
                    Debug.Log("Critical Hit!");
                }

                // Apply the total calculated damage
                enemy.Health -= totalDamage;

                // Apply DOT effect if enabled
                if (swordData.hasDamageOverTime)
                {
                    enemy.ApplyDamageOverTime(swordData.dotDamage, swordData.dotDuration);
                }
            }
        }
    }
}

### END
"""



Exemplars1 = """
### EXAMPLAR 
Example of correct output given an input: 

user prompt = an simple iron sword

output = 
using UnityEngine;

[CreateAssetMenu(fileName = "NewSwordData", menuName = "Sword/Create New Sword Data")]
public class SwordData : ScriptableObject
{
    // Basic stats
    public float damage = 10f;
    public float swingSpeed = 1.0f;
    
    // Damage over time properties
    public bool hasDamageOverTime = false;
    public float dotDamage = 0f;
    public float dotDuration = 0f;
    
    // Elemental bonus damage properties
    public float fireBonusDamage = 0f;
    public float iceBonusDamage = 0f;
    public float lightningBonusDamage = 0f;
    
    // Critical hit properties
    [Range(0f, 1f)]
    public float critChance = 0.1f;       // 10% chance by default
    public float critMultiplier = 2.0f;   // Critical hit deals double damage
    
    // Swing Arc or Range properties
    // swingArc can be used for visual/animation purposes, while colliderSize adjusts the area of effect.
    public float swingArc = 45f;                    // e.g., degrees of the swing arc
    public Vector2 colliderSize = new Vector2(1f, 1f); // for adjusting collider size
}
### END
"""

Exemplars2 = """
### EXAMPLAR 
Example of correct output given an input: 

user prompt = a large two handed greate sword

output = 
using UnityEngine;

[CreateAssetMenu(fileName = "NewSwordData", menuName = "Sword/Create New Sword Data")]
public class SwordData : ScriptableObject
{
    // Basic stats
    public float damage = 10f;
    public float swingSpeed = 1.0f;
    
    // Damage over time properties
    public bool hasDamageOverTime = false;
    public float dotDamage = 0f;
    public float dotDuration = 0f;
    
    // Elemental bonus damage properties
    public float fireBonusDamage = 0f;
    public float iceBonusDamage = 0f;
    public float lightningBonusDamage = 0f;
    
    // Critical hit properties
    [Range(0f, 1f)]
    public float critChance = 0.1f;       // 10% chance by default
    public float critMultiplier = 2.0f;   // Critical hit deals double damage
    
    // Swing Arc or Range properties
    // swingArc can be used for visual/animation purposes, while colliderSize adjusts the area of effect.
    public float swingArc = 45f;                    // e.g., degrees of the swing arc
    public Vector2 colliderSize = new Vector2(1f, 1f); // for adjusting collider size
}
### END
"""
Exemplars3 = """
### EXAMPLAR 
Example of correct output given an input: 

user prompt = quick poison dagger

output = 
using UnityEngine;

[CreateAssetMenu(fileName = "NewSwordData", menuName = "Sword/Create New Sword Data")]
public class SwordData : ScriptableObject
{
    // Basic stats
    public float damage = 4f;
    public float swingSpeed = 2.0f;
    
    // Damage over time properties
    public bool hasDamageOverTime = true;
    public float dotDamage = 1f;
    public float dotDuration = 6f;
    
    // Elemental bonus damage properties
    public float fireBonusDamage = 0f;
    public float iceBonusDamage = 0f;
    public float lightningBonusDamage = 0f;
    
    // Critical hit properties
    [Range(0f, 1f)]
    public float critChance = 0.3f;     
    public float critMultiplier = 2.0f;  
    
    // Swing Arc or Range properties
    // swingArc can be used for visual/animation purposes, while colliderSize adjusts the area of effect.
    public float swingArc = 20f;                    // e.g., degrees of the swing arc
    public Vector2 colliderSize = new Vector2(0.6f, 0.6f); // for adjusting collider size
}
### END
"""

# Conditional_rules = """
# ### CONDITIONAL RULES 

# if the users prompt contains the words:

# large, big, giant, extreme, high damage, massive etc. set: public float damage = 10 in the sword class

# small, fast, short, slim, quick, etc. set: public float damage = 3 in the sword class

# normal, simple, standard, basic, etc. set: public float damage = 6 in the sword class

# """


output_break = """"
### OUTPUT:

"""

In [ ]:
# I made model option within the function, which is probably not the best practice 

import time
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def process_prompts(prompts,
                    GwenOnly = False, StarOnly = False, models7B = False, modelsALL = False, LargeTest = False, Llama = False,
                    prefix = prompt_prefix, Sword_parameter_options = Sword_Item_Parameters,
                    Sword_Template = "", Exemplars = "", 
                    Conditional_rules = "", output = output_break, run = "0" ):
    
    # Define models and their configurations
    models = {}
    
    if LargeTest == True:
        print("Large test models selected") 
        models = {
            "Mixtral": {
                "name": "mistralai/Mixtral-8x7B-v0.1",
                "output_file": f"MixtralOutput_run{run}.txt",
                "time_file": f"MixtralTime_run{run}.txt",
            },
            "Qwen-32B": {
                "name": "Qwen/Qwen2.5-Coder-32B",
                "output_file": f"QwenOutput32Bs_run{run}.txt",
                "time_file": f"QwenTimes32B_run{run}.txt",
            }
            
        }
    elif Llama == True:
        print("llama general text models selected") 
        models = {
            "llama7B": {
                "name": "meta-llama/Llama-2-7b-hf",
                "output_file": f"Lamma7B_Output_run{run}.txt",
                "time_file": f"Lamma7B_Time_run{run}.txt",
            },
            "llama13B": {
                "name": "meta-llama/Llama-2-13b-hf",
                "output_file": f"Lamma13B_Output_run{run}.txt",
                "time_file": f"Lamma13B_Times_run{run}.txt",
            }
            
        }
    elif modelsALL == True & GwenOnly == False & StarOnly == False:
        print("All models selected") 
        models = {
            "Qwen": {
                "name": "Qwen/Qwen2.5-Coder-14B",
                "output_file": f"QwenOutput14Bs_run{run}.txt",
                "time_file": f"QwenTimes14B_run{run}.txt",
            },
            "StarCoder2": {
                "name": "bigcode/starcoder2-15b",
                "output_file": f"StarCoder2Outputs15B_run{run}.txt",
                "time_file": f"StarCoder2Times15B_run{run}.txt",
            },
            "Qwen-7B": {
                "name": "Qwen/Qwen2.5-Coder-7B",
                "output_file": f"QwenOutputs7b_run{run}.txt",
                "time_file": f"QwenTimes7b_run{run}.txt",
            },
            "StarCoder2-7B": {
                "name": "bigcode/starcoder2-7b",
                "output_file": f"StarCoder2Outputs7b_run{run}.txt",
                "time_file": f"StarCoder2Times7b_run{run}.txt",
            }

        }
    elif modelsALL == True & GwenOnly == True:
        print("All Gwen models selected") 
        models = {
            "Qwen": {
                "name": "Qwen/Qwen2.5-Coder-14B",
                "output_file": f"QwenOutput14Bs_run{run}.txt",
                "time_file": f"QwenTimes14B_run{run}.txt",
            },
            "Qwen-7B": {
                "name": "Qwen/Qwen2.5-Coder-7B",
                "output_file": f"QwenOutputs7b_run{run}.txt",
                "time_file": f"QwenTimes7b_run{run}.txt",
            }
        }
    elif modelsALL == True & StarOnly == True:
        print("All Star models selected") 
        models = {
            "StarCoder2": {
                "name": "bigcode/starcoder2-15b",
                "output_file": f"StarCoder2Outputs15B_run{run}.txt",
                "time_file": f"StarCoder2Times15B_run{run}.txt",
            },
            "StarCoder2-7B": {
                "name": "bigcode/starcoder2-7b",
                "output_file": f"StarCoder2Outputs7b_run{run}.txt",
                "time_file": f"StarCoder2Times7b_run{run}.txt",
            }
        }
    elif modelsALL == True & StarOnly == True:
        print("All Star models selected") 
        models = {
            "StarCoder2-7B": {
                "name": "bigcode/starcoder2-7b",
                "output_file": f"StarCoder2Outputs7b_run{run}.txt",
                "time_file": f"StarCoder2Times7b_run{run}.txt",
            },
            "StarCoder2": {
                "name": "bigcode/starcoder2-15b",
                "output_file": f"StarCoder2Outputs15B_run{run}.txt",
                "time_file": f"StarCoder2Times15B_run{run}.txt",
            }

        }
    elif models7B == True & GwenOnly == False:
        print("Small models selected") 
        models = {
            "Qwen-7B": {
                "name": "Qwen/Qwen2.5-Coder-7B",
                "output_file": f"QwenOutputs7b_run{run}.txt",
                "time_file": f"QwenTimes7b_run{run}.txt",
            },
            "StarCoder2-7B": {
                "name": "bigcode/starcoder2-7b",
                "output_file": f"StarCoder2Outputs7b_run{run}.txt",
                "time_file": f"StarCoder2Times7b_run{run}.txt",
            }

        }
    elif models7B == True & GwenOnly == True:
        print("Small Gwen models selected") 
        models = {
            "Qwen-7B": {
                "name": "Qwen/Qwen2.5-Coder-7B",
                "output_file": f"QwenOutputs7b_run{run}.txt",
                "time_file": f"QwenTimes7b_run{run}.txt",
            }
        }
    elif models7B == False & modelsALL == False:
        print("large models selected") 
        
        models = {
            "Qwen": {
                "name": "Qwen/Qwen2.5-Coder-14B",
                "output_file": f"QwenOutput14Bs_run{run}.txt",
                "time_file": f"QwenTimes14B_run{run}.txt",
            },
            "StarCoder2": {
                "name": "bigcode/starcoder2-15b",
                "output_file": f"StarCoder2Outputs15B_run{run}.txt",
                "time_file": f"StarCoder2Times15B_run{run}.txt",
            }
        }
    else:
        print("make a model selection!")
        
        
    # Initialize output storage
    outputs = {key: [] for key in models}
    times = {key: [] for key in models}

    # Process each model
    for model_name, model_info in models.items():
        print(f"Processing prompts for {model_name}...")

        # Load model and tokenizer
        model = AutoModelForCausalLM.from_pretrained(model_info["name"], torch_dtype=torch.float32
                                                     ,device_map="auto"
                                                    )#.to("cuda")
        
        tokenizer = AutoTokenizer.from_pretrained(model_info["name"])
        
        device = model.device  # to make sure the tokinzer is on the same thing -> as i'm using the auto map 

        for prompt in prompts:
            # Start timing
            start_time = time.time()
            
            #total input text section 
            inputtext_instructions = prefix + prompt + Sword_parameter_options + Sword_Template + Exemplars + Conditional_rules + output_break
   
            # Generate output
            inputs = tokenizer(inputtext_instructions, return_tensors="pt").to(device)
            generated_ids = model.generate(**inputs, max_new_tokens=500, repetition_penalty=0.2, do_sample=True, temperature=0.7, top_p=0.9)
            generated_output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

            # End timing
            end_time = time.time()

            # Append results
            outputs[model_name].append((prompt, generated_output))
            times[model_name].append(end_time - start_time)

            # Log the result
            # print(f"Prompt: {prompt}")
            print(f"Time taken: {end_time - start_time:.2f} seconds")
            print(f"Generated output: {generated_output}\n")

        # Save outputs to files
        with open(model_info["output_file"], "w") as output_file:
            for prompt, output in outputs[model_name]:
                output_file.write(f"Prompt: {prompt}\nOutput:\n{output}\n{'-' * 80}\n")

        # Save times to file
        with open(model_info["time_file"], "w") as time_file:
            for prompt, timing in zip(prompts, times[model_name]):
                time_file.write(f"Prompt: {prompt}\nTime: {timing:.2f} seconds\n{'-' * 80}\n")
    
    print("Processing complete.")






In [ ]:
# Input prompts

prompts = [
    "A small iron sword.",
    "A large two-handed sword.",
    "A steel short sword.",
    "A bronze sword.",
    "A black sword with a red gem.",
    "A rusty old sword.",
    "A shining silver sword.",
    "A sword with a leather grip.",
    "A fire sword.",
    "A sword with an icy blue blade.",
    "A knight's sword with a golden hilt.",
    "A plain copper sword.",
    "A curved blade sword.",
    "A sword with a green handle.",
    "A glowing gold sword.",
    "A small dagger-like sword.",
    "A big heavy sword.",
    "A thin, fast sword.",
    "A red sword with jagged edges.",
    "A royal sword with a blue gem."
]

In [ ]:
# Test runs 
# test = ["A small iron sword."] 

# Run config 1: prefix, option 
process_prompts(prompts=test, run="1", LargeTest = False, Llama = True)

# Run config 2: prefix, option, examplar 1 
process_prompts(prompts=prompts, Exemplars=Exemplars1, run="2", LargeTest = False, Llama = True)

# Run config 3: prefix, option, examplar 1, examplar 2
process_prompts(prompts=prompts, Exemplars=(Exemplars1 + Exemplars2), run="3", LargeTest = False, Llama = True)

# Run config 4: prefix, option, examplar 1, examplar 2, examplar 3
process_prompts(prompts=prompts, Exemplars=(Exemplars1 + Exemplars2 + Exemplars3), run="4", LargeTest = False, Llama = True)

# Run config 5: prefix, option, examplar 1, examplar 2, examplar 3, sword template
process_prompts(prompts=prompts, Exemplars=(Exemplars1 + Exemplars2 + Exemplars3), Sword_Template=Sword_Code_Template, run="5", LargeTest = False, Llama = True)

